<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#面向金融领域的NER" data-toc-modified-id="面向金融领域的NER-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>面向金融领域的NER</a></span><ul class="toc-item"><li><span><a href="#数据加载" data-toc-modified-id="数据加载-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>数据加载</a></span><ul class="toc-item"><li><span><a href="#训练集导入" data-toc-modified-id="训练集导入-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>训练集导入</a></span></li><li><span><a href="#测试集导入" data-toc-modified-id="测试集导入-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>测试集导入</a></span></li></ul></li></ul></li><li><span><a href="#全监督下的CRF" data-toc-modified-id="全监督下的CRF-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>全监督下的CRF</a></span><ul class="toc-item"><li><span><a href="#特征选择" data-toc-modified-id="特征选择-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>特征选择</a></span></li><li><span><a href="#参数设定" data-toc-modified-id="参数设定-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>参数设定</a></span></li><li><span><a href="#训练" data-toc-modified-id="训练-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>训练</a></span></li><li><span><a href="#效果测试" data-toc-modified-id="效果测试-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>效果测试</a></span></li><li><span><a href="#结果分析" data-toc-modified-id="结果分析-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>结果分析</a></span></li><li><span><a href="#具体细节" data-toc-modified-id="具体细节-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>具体细节</a></span></li></ul></li><li><span><a href="#弱监督CRF" data-toc-modified-id="弱监督CRF-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>弱监督CRF</a></span></li><li><span><a href="#全监督BiLSTM-CRF的NER" data-toc-modified-id="全监督BiLSTM-CRF的NER-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>全监督BiLSTM-CRF的NER</a></span><ul class="toc-item"><li><span><a href="#基础设定" data-toc-modified-id="基础设定-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>基础设定</a></span></li><li><span><a href="#模型搭建" data-toc-modified-id="模型搭建-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>模型搭建</a></span></li><li><span><a href="#训练" data-toc-modified-id="训练-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>训练</a></span></li></ul></li><li><span><a href="#弱监督BiLSTM-CRF" data-toc-modified-id="弱监督BiLSTM-CRF-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>弱监督BiLSTM-CRF</a></span></li></ul></div>

# 面向金融领域的NER
---
>method：CRF & BiLSTM-CRF

## 数据加载
### 训练集导入

In [1]:
f = open('./data/example.train', 'r',encoding='utf-8')

In [2]:
class Sentence:
    def __init__(self, word_list):
        self.word_list = word_list
        self.score = 0

In [3]:
def load_data(f):
    type_dic = {}
    sent_list = []
    word_list = []
    line = f.readline()
    line_count = 1
    while line:
        line_count += 1
        word_tag = line.strip('\n').split(' ')  # 将句子末尾的换行符删除
        if len(word_tag) > 1:
            type_dic[word_tag[1]] = word_tag[1]
            if word_tag[0] == '。':  # 切分语料，以‘。’为切分节点
                if len(word_list) > 0:
                    word_list.append(word_tag)
                    sent_list.append(Sentence(word_list))
                    word_list = []
            else:
                word_list.append(word_tag)
        line = f.readline()
    f.close()
    return sent_list, line_count, type_dic

In [4]:
data, lines, type_dic = load_data(f)

In [5]:
print('训练集总计数据条数--->',len(data))
type_dic

训练集总计数据条数---> 9039


{'O': 'O',
 'B-yifang': 'B-yifang',
 'I-yifang': 'I-yifang',
 'B-jiafang': 'B-jiafang',
 'I-jiafang': 'I-jiafang',
 'B-xiangmu': 'B-xiangmu',
 'I-xiangmu': 'I-xiangmu',
 'B-moneyDown': 'B-moneyDown',
 'I-moneyDown': 'I-moneyDown',
 'B-hetong': 'B-hetong',
 'I-hetong': 'I-hetong'}

### 测试集导入

In [6]:
test_f=open('./data/example.test','r',encoding='utf-8')
test_data,lines,test_type_dic=load_data(test_f)
print('测试数据集--->',len(test_data))
test_type_dic

测试数据集---> 2799


{'O': 'O',
 'B-jiafang': 'B-jiafang',
 'I-jiafang': 'I-jiafang',
 'B-moneyDown': 'B-moneyDown',
 'I-moneyDown': 'I-moneyDown',
 'B-yifang': 'B-yifang',
 'I-yifang': 'I-yifang',
 'B-xiangmu': 'B-xiangmu',
 'I-xiangmu': 'I-xiangmu',
 'B-hetong': 'B-hetong',
 'I-hetong': 'I-hetong'}

>**金融数据的<span class="mark">实体主要分为5种</span>：**
+ 公司甲方
+ 公司乙方
+ 项目
+ 金额
+ 合同

# 全监督下的CRF

## 特征选择
>主要选择当前字前后两字

In [7]:
def word_features(sent, i):
    word = sent[i][0]
    #postag = sent[i][1]
    features = [
        'bias',
        'word='+word,
    ]

    if i == 0:
        features.append('BOS')

    if i == len(sent)-1:
        features.append('EOS')

    # 控制前后取多少个字作为特征
    n_gram = 2

    temp_n = n_gram
    temp_m = 1
    for n in range(i-n_gram, i):
        if n < 0:
            temp_n -= 1
        else:
            features.append('pre_'+str(temp_n)+'_word='+sent[n][0])
            temp_n -= 1

    for m in range(i+1, i+n_gram+1):
        if(m > len(sent)-1):
            break
        else:
            features.append('next_'+str(temp_m)+'_word='+sent[m][0])
            temp_m += 1

    return features

In [8]:
def sent_features(sent):
    return [word_features(sent, i) for i in range(len(sent))]


def sent_r_label(sent):
    return [r_label for word, r_label, p_label in sent]


def sent_p_label(sent):
    return [p_label for word, r_label, p_label in sent]


def sent_word(sent):
    return [word for word, r_label, p_label in sent]


def sent_word2(sent):
    return [word for word, label in sent]


def sent_label(sent):
    return [label for word, label in sent]

In [11]:
print(sent_features(data[1].word_list))

[['bias', 'word=现', 'BOS', 'next_1_word=将', 'next_2_word=中'], ['bias', 'word=将', 'pre_1_word=现', 'next_1_word=中', 'next_2_word=标'], ['bias', 'word=中', 'pre_2_word=现', 'pre_1_word=将', 'next_1_word=标', 'next_2_word=的'], ['bias', 'word=标', 'pre_2_word=将', 'pre_1_word=中', 'next_1_word=的', 'next_2_word=有'], ['bias', 'word=的', 'pre_2_word=中', 'pre_1_word=标', 'next_1_word=有', 'next_2_word=关'], ['bias', 'word=有', 'pre_2_word=标', 'pre_1_word=的', 'next_1_word=关', 'next_2_word=内'], ['bias', 'word=关', 'pre_2_word=的', 'pre_1_word=有', 'next_1_word=内', 'next_2_word=容'], ['bias', 'word=内', 'pre_2_word=有', 'pre_1_word=关', 'next_1_word=容', 'next_2_word=公'], ['bias', 'word=容', 'pre_2_word=关', 'pre_1_word=内', 'next_1_word=公', 'next_2_word=告'], ['bias', 'word=公', 'pre_2_word=内', 'pre_1_word=容', 'next_1_word=告', 'next_2_word=如'], ['bias', 'word=告', 'pre_2_word=容', 'pre_1_word=公', 'next_1_word=如', 'next_2_word=下'], ['bias', 'word=如', 'pre_2_word=公', 'pre_1_word=告', 'next_1_word=下', 'next_2_word=:'], ['bias',

In [14]:
x_train = [sent_features(s.word_list) for s in data]
y_train = [sent_label(s.word_list) for s in data]

In [15]:
import pycrfsuite as pycrf

In [29]:
%%time
trainer = pycrf.Trainer(verbose=True)

for x_seq, y_seq in zip(x_train, y_train):
    trainer.append(x_seq, y_seq)

Wall time: 4.54 s


## 参数设定

In [31]:
trainer.set_params({
    'c1': 1.0,
    'c2': 1e-3,
    'max_iterations': 50,
    'feature.possible_transitions': True
})

## 训练

In [32]:
%%time
trainer.train('./ner_model/supervised_crf_model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 36885
Seconds required: 1.039

L-BFGS optimization
c1: 1.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 1010162.333099
Feature norm: 1.000000
Error norm: 833721.347904
Active features: 30740
Line search trials: 1
Line search step: 0.000001
Seconds required for this iteration: 1.152

***** Iteration #2 *****
Loss: 702547.929703
Feature norm: 5.105485
Error norm: 159064.997063
Active features: 30131
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 1.310

***** Iteration #3 *****
Loss: 610480.928047
Feature norm: 4.554094
Error norm: 158468.462837
Active features: 25626
Line search trials: 1
Line search step: 1.000000
Seconds requir

***** Iteration #46 *****
Loss: 25270.863619
Feature norm: 31.794235
Error norm: 6112.003003
Active features: 27756
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.666

***** Iteration #47 *****
Loss: 23874.107747
Feature norm: 32.902897
Error norm: 6441.033597
Active features: 27417
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.582

***** Iteration #48 *****
Loss: 23262.649255
Feature norm: 35.163546
Error norm: 22110.360959
Active features: 26802
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.649

***** Iteration #49 *****
Loss: 22181.389115
Feature norm: 36.191747
Error norm: 4158.866458
Active features: 26956
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.611

***** Iteration #50 *****
Loss: 21534.041129
Feature norm: 37.368756
Error norm: 7639.801394
Active features: 26723
Line search trials: 1
Line search step: 1.00000

## 效果测试

In [25]:
test_f=open('./data/example.test','r',encoding='utf-8')
test_data,lines,test_type_dic=load_data(test_f)
print('测试数据集--->',len(test_data))
test_type_dic

测试数据集---> 2799


{'O': 'O',
 'B-jiafang': 'B-jiafang',
 'I-jiafang': 'I-jiafang',
 'B-moneyDown': 'B-moneyDown',
 'I-moneyDown': 'I-moneyDown',
 'B-yifang': 'B-yifang',
 'I-yifang': 'I-yifang',
 'B-xiangmu': 'B-xiangmu',
 'I-xiangmu': 'I-xiangmu',
 'B-hetong': 'B-hetong',
 'I-hetong': 'I-hetong'}

In [34]:
tagger=pycrf.Tagger()
tagger.open('./ner_model/supervised_crf_model')

In [44]:
print(' '.join(sent_word2(test_data[8].word_list)), '\n')
print('predict_label--->',
      ' '.join(tagger.tag(sent_features(test_data[8].word_list))), '\n')
print('real_label--->', ' '.join(sent_label(test_data[8].word_list)))

在 国 家 电 网 公 司 2 0 1 6 年 第 四 批 信 息 化 项 目 ( 设 备 、 调 度 控 制 系 统 ) 招 标 采 购 — 调 度 类 硬 件 ( 招 标 编 号 : 0 7 1 1 - 1 6 O T L 1 2 5 1 2 0 0 3 ) 项 目 中 , 公 司 共 中 4 个 包 , 分 别 为 : 包 2 、 包 1 2 、 包 2 2 和 包 2 3 , 金 额 为 3 0 1 6 7 4 0 0 。 

predict_label---> O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-moneyDown I-moneyDown I-moneyDown I-moneyDown I-moneyDown I-moneyDown I-moneyDown I-moneyDown O 

real_label---> O B-jiafang I-jiafang I-jiafang I-jiafang I-jiafang I-jiafang O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-moneyDown I-moneyDown I-moneyDown I-moneyDown I-moneyDown I-moneyDown I-moneyDown I-moneyDown O


In [23]:
from itertools import chain
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn

In [24]:
def bio_classification_report(y_true, y_pred):

    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))

    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}

    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels=[class_indices[cls] for cls in tagset],
        target_names=tagset,
    )

In [49]:
y_test = [sent_label(s.word_list) for s in test_data]
x_test = [sent_features(s.word_list) for s in test_data]
p_test = [tagger.tag(s) for s in x_test]
if len(y_test) != len(p_test):
    print('length error!')

In [50]:
print(bio_classification_report(y_test, p_test))

             precision    recall  f1-score   support

   B-hetong       0.70      0.45      0.55       176
   I-hetong       0.66      0.45      0.54      2977
  B-jiafang       0.61      0.44      0.51       487
  I-jiafang       0.59      0.49      0.54      5400
B-moneyDown       0.56      0.25      0.35       340
I-moneyDown       0.54      0.26      0.35      2663
  B-xiangmu       0.39      0.15      0.22       305
  I-xiangmu       0.43      0.23      0.30      7690
   B-yifang       0.84      0.75      0.79       695
   I-yifang       0.84      0.78      0.81      7998

avg / total       0.62      0.47      0.53     28731



## 结果分析
>在crf中，效果不是很好
>><span class="girk">对项目的识别效果最差</span>
---

## 具体细节

In [51]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("最有可能的组合:")
print_transitions(Counter(info.transitions).most_common(10))

print("\n最不可能的组合:")
print_transitions(Counter(info.transitions).most_common()[-10:])

最有可能的组合:
I-xiangmu -> I-xiangmu 5.246446
I-yifang -> I-yifang 5.067625
O      -> O       3.656521
I-jiafang -> I-jiafang 3.599144
I-hetong -> I-hetong 3.258706
B-yifang -> I-yifang 3.184577
B-xiangmu -> I-xiangmu 2.737900
B-hetong -> I-hetong 2.545955
I-moneyDown -> I-moneyDown 2.203954
B-jiafang -> I-jiafang 1.742877

最不可能的组合:
I-yifang -> I-jiafang -5.285346
B-yifang -> O       -5.530258
I-xiangmu -> I-jiafang -5.692742
O      -> I-xiangmu -6.073576
O      -> I-yifang -6.189786
B-jiafang -> O       -6.251360
B-xiangmu -> O       -6.376273
O      -> I-moneyDown -6.855241
O      -> I-hetong -7.439036
O      -> I-jiafang -7.500449


# 弱监督CRF
---
>初始种子训练集3000，迭代步长1000

In [16]:
seed, step = 3000, 1000

In [18]:
seed_x_train = x_train[:seed]
seed_y_train = y_train[:seed]

In [22]:
loop = 0
trainer = pycrf.Trainer(verbose=True)
trainer.set_params({
    'c1': 1.0,
    'c2': 1e-3,
    'max_iterations': 50,
    'feature.possible_transitions': True
})
tagger = pycrf.Tagger()
new_x_train = []
new_y_train = []

while (seed+step*loop) < len(x_train):

    if loop == 0:
        new_x_train = seed_x_train
        new_y_train = seed_y_train

    for x_seq, y_seq in zip(new_x_train, new_y_train):
        trainer.append(x_seq, y_seq)
    new_x_train=[] #清空
    new_y_train=[]
    tagger_name = './ner_model/weakly_supervised_crf_model'+str(loop)
    trainer.train(tagger_name)  # 自动保存

    tagger.open(tagger_name)
    
    new_x_train=x_train[seed+loop*step:seed+(loop+1)*step]
    for new_item in new_x_train:
        new_y_train.append(tagger.tag(new_item))
    loop += 1

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 25156
Seconds required: 0.491

L-BFGS optimization
c1: 1.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 339208.781704
Feature norm: 1.000000
Error norm: 282656.317774
Active features: 20235
Line search trials: 1
Line search step: 0.000003
Seconds required for this iteration: 0.391

***** Iteration #2 *****
Loss: 231064.952358
Feature norm: 5.121583
Error norm: 52077.268556
Active features: 19883
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.408

***** Iteration #3 *****
Loss: 201608.909406
Feature norm: 4.582697
Error norm: 51890.018975
Active features: 16836
Line search trials: 1
Line search step: 1.000000
Seconds required 

***** Iteration #48 *****
Loss: 7317.619532
Feature norm: 37.435518
Error norm: 4901.863693
Active features: 16164
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.216

***** Iteration #49 *****
Loss: 7079.852919
Feature norm: 38.474343
Error norm: 2197.784460
Active features: 15954
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.201

***** Iteration #50 *****
Loss: 6817.367405
Feature norm: 39.949718
Error norm: 2160.661242
Active features: 15403
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.222

L-BFGS terminated with the maximum number of iterations
Total seconds required for training: 17.026

Storing the model
Number of active features: 15403 (25156)
Number of active attributes: 5930 (10615)
Number of active labels: 11 (11)
Writing labels
Writing attributes
Writing feature references for transitions
Writing feature references for attributes
Seconds required: 0.01

***** Iteration #34 *****
Loss: 20110.607243
Feature norm: 19.775834
Error norm: 18069.704779
Active features: 19139
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.239

***** Iteration #35 *****
Loss: 19653.316766
Feature norm: 21.674825
Error norm: 41047.403232
Active features: 19017
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.248

***** Iteration #36 *****
Loss: 18540.703839
Feature norm: 23.294647
Error norm: 19429.462939
Active features: 19069
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.249

***** Iteration #37 *****
Loss: 18145.842282
Feature norm: 25.569337
Error norm: 41286.645688
Active features: 19002
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.261

***** Iteration #38 *****
Loss: 16918.030713
Feature norm: 26.576285
Error norm: 15786.909343
Active features: 19081
Line search trials: 1
Line search step: 1.0

***** Iteration #30 *****
Loss: 26796.568689
Feature norm: 17.151344
Error norm: 14103.109103
Active features: 20528
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.336

***** Iteration #31 *****
Loss: 26239.209775
Feature norm: 18.014249
Error norm: 54088.136942
Active features: 20315
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.680

***** Iteration #32 *****
Loss: 25031.751060
Feature norm: 18.589018
Error norm: 26442.988177
Active features: 20372
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.396

***** Iteration #33 *****
Loss: 24657.358433
Feature norm: 19.938078
Error norm: 63909.378991
Active features: 20244
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.579

***** Iteration #34 *****
Loss: 23060.808572
Feature norm: 20.820654
Error norm: 26948.432416
Active features: 20406
Line search trials: 1
Line search step: 1.0

***** Iteration #16 *****
Loss: 54899.237653
Feature norm: 10.396175
Error norm: 17394.908153
Active features: 23079
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.434

***** Iteration #17 *****
Loss: 52919.296423
Feature norm: 10.757516
Error norm: 16687.015789
Active features: 22940
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.473

***** Iteration #18 *****
Loss: 50849.673057
Feature norm: 11.801583
Error norm: 31076.831745
Active features: 22012
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.628

***** Iteration #19 *****
Loss: 46957.207576
Feature norm: 12.484040
Error norm: 31289.931579
Active features: 21232
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.452

***** Iteration #20 *****
Loss: 42064.221834
Feature norm: 13.474078
Error norm: 30869.374568
Active features: 21114
Line search trials: 1
Line search step: 1.0

***** Iteration #11 *****
Loss: 75107.371280
Feature norm: 8.687136
Error norm: 53291.496933
Active features: 24436
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 1.009

***** Iteration #12 *****
Loss: 69228.750698
Feature norm: 8.974801
Error norm: 26328.649508
Active features: 24354
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.512

***** Iteration #13 *****
Loss: 67345.177564
Feature norm: 9.410331
Error norm: 71682.914369
Active features: 24218
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 1.099

***** Iteration #14 *****
Loss: 65170.766122
Feature norm: 9.448263
Error norm: 15573.853103
Active features: 24485
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.825

***** Iteration #15 *****
Loss: 63193.750494
Feature norm: 9.722426
Error norm: 12185.310246
Active features: 24324
Line search trials: 1
Line search step: 1.000000

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 31692
Seconds required: 0.850

L-BFGS optimization
c1: 1.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 862363.954650
Feature norm: 1.000000
Error norm: 777358.362980
Active features: 26815
Line search trials: 1
Line search step: 0.000001
Seconds required for this iteration: 1.069

***** Iteration #2 *****
Loss: 848078.509333
Feature norm: 9.608574
Error norm: 103398.319432
Active features: 26245
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.517

***** Iteration #3 *****
Loss: 755646.795171
Feature norm: 8.746933
Error norm: 103394.215572
Active features: 21251
Line search trials: 1
Line search step: 1.000000
Seconds require

***** Iteration #42 *****
Loss: 13367.271702
Feature norm: 28.911778
Error norm: 4258.223285
Active features: 20903
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.518

***** Iteration #43 *****
Loss: 12938.777537
Feature norm: 29.456146
Error norm: 6754.957729
Active features: 20466
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.548

***** Iteration #44 *****
Loss: 12378.788023
Feature norm: 30.204420
Error norm: 7357.909698
Active features: 19977
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.529

***** Iteration #45 *****
Loss: 12023.396393
Feature norm: 31.574621
Error norm: 22914.261559
Active features: 19093
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.550

***** Iteration #46 *****
Loss: 11635.521732
Feature norm: 31.894406
Error norm: 2666.591212
Active features: 19583
Line search trials: 1
Line search step: 1.00000

***** Iteration #39 *****
Loss: 23177.885633
Feature norm: 30.461439
Error norm: 44502.031320
Active features: 22651
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.584

***** Iteration #40 *****
Loss: 21856.922420
Feature norm: 30.647303
Error norm: 16570.785818
Active features: 22682
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.652

***** Iteration #41 *****
Loss: 21101.037818
Feature norm: 30.980462
Error norm: 36670.974414
Active features: 22588
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.646

***** Iteration #42 *****
Loss: 20095.299048
Feature norm: 31.277776
Error norm: 21289.337288
Active features: 22558
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.588

***** Iteration #43 *****
Loss: 19629.889518
Feature norm: 31.777953
Error norm: 46274.247417
Active features: 22460
Line search trials: 1
Line search step: 1.0

In [26]:
tagger.open('./ner_model/weakly_supervised_crf_model6')
y_test = [sent_label(s.word_list) for s in test_data]
x_test = [sent_features(s.word_list) for s in test_data]
p_test = [tagger.tag(s) for s in x_test]
if len(y_test) != len(p_test):
    print('length error!')

In [27]:
print(bio_classification_report(y_test, p_test))

             precision    recall  f1-score   support

   B-hetong       0.78      0.22      0.35       176
   I-hetong       0.72      0.14      0.23      2977
  B-jiafang       0.61      0.28      0.39       487
  I-jiafang       0.63      0.30      0.41      5400
B-moneyDown       0.64      0.12      0.20       340
I-moneyDown       0.62      0.13      0.21      2663
  B-xiangmu       0.47      0.08      0.14       305
  I-xiangmu       0.47      0.13      0.20      7690
   B-yifang       0.81      0.56      0.66       695
   I-yifang       0.82      0.58      0.68      7998

avg / total       0.65      0.30      0.39     28731



# 全监督BiLSTM-CRF的NER

## 基础设定

In [22]:
#将字向量转换成100维
EMBED_DIM = 100
BiRNN_UNITS = 200

In [9]:
import numpy
from collections import Counter
from keras.preprocessing.sequence import pad_sequences
import pickle
import platform

In [16]:
#字统计
word_counts = Counter(row[0] for sample in data for row in sample.word_list)#字典，用于onehot编码
vocab = [w for w, f in iter(word_counts.items()) if f >= 2]

In [17]:
chunk_tags=[key for key in type_dic]
chunk_tags

['O',
 'B-yifang',
 'I-yifang',
 'B-jiafang',
 'I-jiafang',
 'B-xiangmu',
 'I-xiangmu',
 'B-moneyDown',
 'I-moneyDown',
 'B-hetong',
 'I-hetong']

In [59]:
# save config data
with open('./ner_model/finance.pkl', 'wb') as outp:
    pickle.dump((vocab, chunk_tags), outp)

In [42]:
def process_data(data, vocab, chunk_tags, maxlen=None, onehot=False):
    if maxlen is None:
        maxlen = max(len(s.word_list) for s in data)
        #print('max length--->', maxlen,'\n')

    word2idx = dict((w, i) for i, w in enumerate(vocab))

    # set to <unk> (index 1) if not in vocab
    x = [[word2idx.get(w[0], 1) for w in s.word_list] for s in data]

    y_chunk = [[chunk_tags.index(w[1]) for w in s.word_list] for s in data]

    x = pad_sequences(x, maxlen)  # left padding
    y_chunk = pad_sequences(y_chunk, maxlen, value=-1)

    if onehot:
        y_chunk = numpy.eye(len(chunk_tags), dtype='float32')[y_chunk]
    else:
        y_chunk = numpy.expand_dims(y_chunk, 2)

    return x, y_chunk,maxlen

## 模型搭建

In [1]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM
from keras_contrib.layers import CRF

D:\software\anaconda3.5\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [44]:
x_lstm,y_lstm,maxlen=process_data(data,vocab,chunk_tags)
print('sentence max length--->',maxlen)

sentence max length---> 2190


In [37]:
count=0
for s in data:
    if len(s.word_list)>300:
        count+=1
print(count)

240


In [23]:
model=Sequential()
# Random embedding
model.add(Embedding(len(vocab), EMBED_DIM, mask_zero=True))
model.add(Bidirectional(LSTM(BiRNN_UNITS // 2, return_sequences=True)))
crf = CRF(len(chunk_tags), sparse_target=True)
model.add(crf)
model.compile('adam', loss=crf.loss_function, metrics=[crf.accuracy])
model.summary()

D:\software\anaconda3.5\lib\site-packages\keras_contrib\layers\crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
D:\software\anaconda3.5\lib\site-packages\keras_contrib\layers\crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         232000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 200)         160800    
_________________________________________________________________
crf_2 (CRF)                  (None, None, 11)          2354      
Total params: 395,154
Trainable params: 395,154
Non-trainable params: 0
_________________________________________________________________


## 训练

In [67]:
EPOCHS = 10
model.fit(x_lstm, y_lstm, batch_size=16, epochs=EPOCHS,validation_split=0.15)
model.save('./ner_model/finance_bilstm_crf.h5')

Train on 7231 samples, validate on 1808 samples
Epoch 1/1
 352/7231 [>.............................] - ETA: 27:14 - loss: 134.6482 - crf_viterbi_accuracy: 0.7112

KeyboardInterrupt: 

In [24]:
model.load_weights('./ner_model/finance_bilstm_crf1200.h5')

In [25]:
import numpy as np

In [49]:
def process_text(data, vocab, maxlen=200):
    word2idx = dict((w, i) for i, w in enumerate(vocab))
    x = [word2idx.get(w[0], 1) for w in data]
    length = len(x)
    x = pad_sequences([x], maxlen)  # left padding
    return x, length

In [27]:
text=test_data[202].word_list
predict_text=''
for i in text:
    predict_text+=i[0]

In [28]:
predict_text

'项目总投资额度不低于人民币1500000000。'

In [29]:
str, length = process_text(predict_text, vocab)
raw = model.predict(str)[0][-length:]

In [30]:
def tag_format(predict_text, result_tags):
    jiafang, yifang, xiangmu, hetong, money = '', '', '', '', ''
    for s, t in zip(predict_text, result_tags):
        if t in ('B-jiafang', 'I-jiafang'):
            jiafang += ' ' + s if (t == 'B-jiafang') else s
        if t in ('B-yifang', 'I-yifang'):
            yifang += ' ' + s if (t == 'B-yifang') else s
        if t in ('B-xiangmu', 'I-xiangmu'):
            xiangmu += ' ' + s if (t == 'B-xiangmu') else s
        if t in ('B-hetong', 'I-hetong'):
            hetong += ' ' + s if (t == 'B-hetong') else s
        if t in ('B-moneyDown', 'I-moneyDown'):
            money += ' ' + s if (t == 'B-moneyDown') else s

    print(['甲方:' + jiafang, '乙方:' + yifang, '项目:' +
           xiangmu, '合同:'+hetong, '金额:'+money])

In [33]:
result = [np.argmax(row) for row in raw]
result_tags = [chunk_tags[i] for i in result]
print('predict label--->')
tag_format(predict_text, result_tags)
print('real label--->')
tag_format(predict_text,sent_label(text))

predict label--->
['甲方:', '乙方:', '项目:', '合同:', '金额:']
real label--->
['甲方:', '乙方:', '项目:', '合同:', '金额: 1500000000']


In [45]:
test_p=[]

In [47]:
%%time
for d in test_data:
    if len(d.word_list)>200:
        continue
    str_arr, length = process_text(d.word_list, vocab)
    tag_raw = model.predict(str_arr)[0][-length:]
    label_result = [np.argmax(row) for row in tag_raw]
    tag_result = [chunk_tags[i] for i in label_result]
    test_p.append(tag_result)

Wall time: 55.5 s


In [50]:
y_test=[sent_label(s.word_list) for s in test_data if len(s.word_list)<=200]

In [57]:
tt_p=[]
tt_y=[]
for x,y in zip(test_p,y_test):
    if len(x)!=len(y):
        continue;
    else:
        tt_p.append(x)
        tt_y.append(y)
len(tt_p)

2003

In [58]:
print(bio_classification_report(tt_p,tt_y))

             precision    recall  f1-score   support

   B-hetong       0.34      0.71      0.46        31
   I-hetong       0.49      0.74      0.59       658
  B-jiafang       0.45      0.74      0.56        57
  I-jiafang       0.50      0.79      0.61       647
B-moneyDown       0.06      0.71      0.11         7
I-moneyDown       0.05      0.71      0.10        51
  B-xiangmu       0.40      0.78      0.53        32
  I-xiangmu       0.51      0.66      0.57       981
   B-yifang       0.76      0.84      0.80       158
   I-yifang       0.90      0.88      0.89      1959

avg / total       0.67      0.79      0.72      4581

